In [4]:
import unittest

import numpy as np

grid = [[0, 1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0]]
goal = [len(grid)-1, len(grid[0])-1]
cost = 1 # the cost associated with moving from a cell to an adjacent one

delta = [[-1, 0 ], # go up
         [ 0, -1], # go left
         [ 1, 0 ], # go down
         [ 0, 1 ]] # go right

delta_name = ['^', '<', 'v', '>']

In [67]:
# list of open nodes, w/ from (value, (coords)) and sorted to show lowest value first
# pop off the first one, and
# check for validity, and if valid
# expand =
#     - add it to the value graph
#     - add adjacent open nodes with appropriate value
# iterate unti no more open nodes

def add_tuples(a,b):
    return (a[0] + b[0], a[1] + b[1])


class Map:
    def __init__(self, grid, goal, cost):
        self.grid = grid
        self.goal = tuple(goal)
        self.cost = cost
        self.dims = self.grid.shape
        
        self.value = np.full(self.grid.shape, 99)
        self.open = [(0, self.goal)]
        
        self.delta = [[-1, 0 ], # go up
                      [ 0, -1], # go left
                      [ 1, 0 ], # go down
                      [ 0, 1 ]] # go right


    def valid_loc(self, loc):
        x,y = loc
        return (0 <= x < self.dims[0] and 0 <= y < self.dims[1] and self.grid[loc] == 0)
    
    
    def next_expand(self, iters=1):
        for i in range(iters):
            if len(self.open) == 0:
                return

            self.open.sort()
            d, loc = self.open.pop(0)

            if self.value[loc] != 99:
                continue

            self.value[loc] = d

            for move in self.delta:
                new_loc = add_tuples(loc, move)

                if self.valid_loc(new_loc) and self.value[new_loc] == 99:
                    self.open.append((d + self.cost, new_loc))
                    
                
    def iter_until_solved(self):
        while len(self.open) > 0:
            self.next_expand()
    
    
class DynamicProgTests(unittest.TestCase):
    
    def setUp(self):
        self.grid = np.array([[0, 1, 0, 0, 0, 0],
                              [0, 1, 0, 0, 0, 0],
                              [0, 1, 0, 0, 0, 0],
                              [0, 1, 0, 0, 0, 0],
                              [0, 0, 0, 0, 1, 0]])
        self.goal = (len(self.grid)-1, len(self.grid[0])-1)
        self.cost = 1
        self.m = Map(self.grid, self.goal, self.cost)
    
    
    def test_initialization(self):
        self.assertEqual(self.m.grid.shape, (5,6))
        self.assertEqual(self.m.value.shape, self.m.grid.shape)
        self.assertEqual(self.m.value[(1,1)], 99)

        
    def test_next_expand(self):
        self.m.next_expand()
        self.assertEqual(self.m.value[self.m.goal], 0)
        
        self.m.next_expand()
        self.assertEqual(self.m.value[(3,5)], 1)
        
        self.m.iter_until_solved()
        expected_value = np.array([[11, 99,  7,  6,  5,  4],
                                   [10, 99,  6,  5,  4,  3],
                                   [ 9, 99,  5,  4,  3,  2],
                                   [ 8, 99,  4,  3,  2,  1],
                                   [ 7,  6,  5,  4, 99,  0]])
        self.assertTrue(np.array_equal(self.m.value, expected_value))
        

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit = False)

..
----------------------------------------------------------------------
Ran 2 tests in 0.003s

OK
